In [34]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model



In [35]:
#%pip install keras-facenet



In [36]:
#%pip install scipy

In [37]:
from keras_facenet import FaceNet
# Création d'une instance FaceNet
embedder = FaceNet()

In [38]:
# Parcourir le dataset et encoder les visages
database_of_known_faces = {}
dataset_path = './dataset'
personnes = os.listdir(dataset_path)
print(personnes)

['Amina', 'Athman', 'Houssine', 'Khalil', 'Nicolas', 'Rachida', 'Sofiane', 'Thibaut', 'Vincent', 'Yassine']


In [39]:
# Charger le modèle FaceNet
#facenet_model = load_model('./facenet_keras.h5')
#print('Loaded Model')

In [40]:
def encode_face(face):
    # Assurez-vous que l'image est en couleur (RGB)
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    # FaceNet s'attend à recevoir une liste d'images, même pour une seule image
    detections = embedder.embeddings([face_rgb])
    
    # Si detections contient des données, retournez-les
    if len(detections) > 0:
        return detections[0]
    else:
        return None


In [ ]:
# def preprocess_face(face):
#     # Redimensionner l'image pour qu'elle corresponde à l'entrée attendue par FaceNet
#     face = cv2.resize(face, (160, 160))
#     # Normaliser les pixels
#     face = face.astype('float32')
#     mean, std = face.mean(), face.std()
#     face = (face - mean) / std
#     # Étendre les dimensions pour correspondre à l'entrée du modèle
#     face = np.expand_dims(face, axis=0)
#     return face

In [42]:

for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    encodings = []
    
    for image_name in os.listdir(person_path):
        image_path = os.path.join(person_path, image_name)
        image = cv2.imread(image_path)
        
        if image is not None:
            encoding = encode_face(image)
            if encoding is not None:
                encodings.append(encoding)
    
    if encodings:
        database_of_known_faces[person_name] = encodings
# À ce stade, database_of_known_faces contient les encodages pour chaque image de chaque individu        

1/1 [==============================] - 0s 218ms/step


In [43]:

# Parcourir les dossiers et les images
# for person_name in os.listdir(dataset_path):
#     person_path = os.path.join(dataset_path, person_name)
#     database_of_known_faces[person_name] = []

#     for image_name in os.listdir(person_path):
#         image_path = os.path.join(person_path, image_name)
#         image = cv2.imread(image_path)
#         face_encoding = encode_face(image)
        
#         database_of_known_faces[person_name].append(face_encoding)

# À ce stade, database_of_known_faces contient les encodages pour chaque image de chaque individu


In [44]:
from scipy.spatial.distance import cosine

def compare_faces(embedding, database):
    min_distance = float('inf')
    name_closest_match = None

    for (name, db_embeddings) in database.items():
        for db_embedding in db_embeddings:
            dist = cosine(embedding, db_embedding)
            if dist < min_distance:
                min_distance = dist
                name_closest_match = name

    # Vous pouvez définir un seuil pour la distance, si c'est au-dessus du seuil, considérez que le visage n'est pas reconnu
    if min_distance > 0.5:  # Le seuil dépend de la précision de votre modèle et de la qualité des encodages
        return None
    return name_closest_match


In [45]:
def detect_face(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Si au moins un visage est détecté, renvoyez les coordonnées du premier visage
    if len(faces) > 0:
        # Renvoie les coordonnées du premier visage
        x, y, w, h = faces[0]
        # Retourne également la région de visage découpée de la frame originale
        return (x, y, w, h), frame[y:y+h, x:x+w]
    else:
        return None, None


In [46]:
video_capture = cv2.VideoCapture(0)
# Capture la frame
# Boucle de capture vidéo
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Erreur de capture")
        break
# la fonction detect_face pour renvoie un tuple (face_location, face_image)
face_location, face_image = detect_face(frame)

# Si un visage est détecté
if face_location is not None:
    #  coordonnées x, y, w, h et l'image du visage
    encoding = encode_face(face_image)  
    x, y, w, h = face_location  # Extraction des coordonnées et dimensions du visage
    # 
    name = compare_faces(encoding, database_of_known_faces)  # Cette fonction doit comparer l'encodage et renvoyer le nom correspondant

    # Affiche le nom ou "Personne inconnue" au-dessus du rectangle du visage
    display_name = name if name else "Personne inconnue"
    cv2.putText(frame, display_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('Video', frame)
    # Quitter le flux vidéo avec 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libération des ressources
video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 